# <font color='green'>PROJETO DE DATA SCIENCE</font>
## <font color='green'>ANÁLISE DE DADOS</font>
### <font color='green'>BANCO DE DADOS RELACIONAL - LINGUAGEM PYTHON E SQL PARA ANÁLISE DE DADOS</font>

Este é um projeto de análise de dados utilizando linguagem de programção Python e SQL em um Jupyter Notebook para analisar um conjunto de dados fictícios de uma rede de varejo em um banco de dados relacional (SQLite).

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('A verão python utilizada neste jupyter notebook:', python_version())

A verão python utilizada neste jupyter notebook: 3.11.5


In [2]:
# Gravar versões de pacotes
#!pip install -q -U watermark

In [3]:
# Pacote para uso de SQL
#!pip install -q ipython-sql

## Instalar e Carregar Pacotes

In [4]:
# Imports
import pandas as pd
import sqlite3

In [5]:
# Versões dos pacotes
%reload_ext watermark
%watermark --iversions

pandas : 2.0.3
sqlite3: 2.6.0



## Banco Relacional, Python e SQL para Análise de Dados

In [6]:
# Criar conexão com o SQLite
cnn = sqlite3.connect('banco_de_dados.db')

In [7]:
# Criar dataframe cliente
df_cliente = pd.read_csv('datasets/cliente.csv')

In [8]:
# Criar uma tabela cliente no SQLite
df_cliente.to_sql('cliente', cnn, if_exists='replace', index=False)

50

In [9]:
# Primeiro - carregar a extensão 
%load_ext sql

In [10]:
# Segundo - criar a conexão com o banco de dados SQLite
%sql sqlite:///banco_de_dados.db

In [11]:
%%sql
--Testar uma consulta
SELECT * FROM cliente LIMIT 3;

 * sqlite:///banco_de_dados.db
Done.


clienteid,nome,endereco,cidadecliente,paiscliente
1,Cliente 1,"Rua R, 29",Buenos Aires,Argentina
2,Cliente 2,"Rua M, 325",São Paulo,Brasil
3,Cliente 3,"Rua G, 627",New York,EUA


In [12]:
# Criar dataframes
df_data = pd.read_csv('datasets/data.csv')
df_distribuidor = pd.read_csv('datasets/distribuidor.csv')
df_produto = pd.read_csv('datasets/produto.csv')
df_vendas = pd.read_csv('datasets/vendas.csv')

In [13]:
dataframes = {
    'data': df_data,
    'distribuidor': df_distribuidor,
    'produto': df_produto,
    'vendas': df_vendas }

In [14]:
# Anexar cada dataframe ao SQLite
for table_name, df in dataframes.items():
    df.to_sql(table_name, cnn, if_exists='append', index=False)

In [15]:
%%sql
--Testar nova consulta
SELECT * FROM data LIMIT 3;

 * sqlite:///banco_de_dados.db
Done.


dataid,data,dia,mes,ano,diadasemana
1,2021-01-01,1,1,2021,Friday
2,2021-01-02,2,1,2021,Saturday
3,2021-01-03,3,1,2021,Sunday


In [16]:
%%sql
---Listar todas tabelas no SQLite
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///banco_de_dados.db
Done.


name
cliente
data
distribuidor
produto
vendas


## RESPONDER PERGUNTAS DE NEGÓCIOS DE UMA REDE DE VAREJO

### 1. Qual o número total de vendas e a média de quantidade vendida?

In [17]:
%%sql
SELECT COUNT(quantidadevendida) AS total_vendas, ROUND(AVG(quantidadevendida), 2) AS media_vendas
FROM vendas;

 * sqlite:///banco_de_dados.db
Done.


total_vendas,media_vendas
10000,50.54


### 2. Qual o número total de produtos únicos vendidos?

In [18]:
%%sql
SELECT COUNT(DISTINCT produtoid) AS Produtos_unicos
FROM vendas;

 * sqlite:///banco_de_dados.db
Done.


Produtos_unicos
100


### 3. Quais os cinco produtos com maior número de vendas?

In [19]:
%%sql
SELECT p.nome AS nome_produto, COUNT(v.quantidadevendida) AS total_vendas
FROM Produto AS p
JOIN Vendas AS v ON v.produtoid = p.produtoid
GROUP BY p.nome
ORDER BY total_vendas DESC
LIMIT 5;

 * sqlite:///banco_de_dados.db
Done.


nome_produto,total_vendas
Produto 48,123
Produto 19,122
Produto 34,119
Produto 97,118
Produto 96,118


### 4. Quais clientes fizeram duzentas e vinte ou mais transações de compra?

In [20]:
%%sql
SELECT c.nome AS nome_cliente, COUNT(c.clienteid) AS total_transacoes
FROM cliente AS c
JOIN vendas AS V ON v.clienteid = c.clienteid
GROUP BY nome_cliente
HAVING COUNT(c.clienteid) >= 220
ORDER BY total_transacoes DESC;

 * sqlite:///banco_de_dados.db
Done.


nome_cliente,total_transacoes
Cliente 11,230
Cliente 2,228
Cliente 37,222
Cliente 33,221


### 5. Listar o nome dos dez produtos com maiores preços da categoria 2.

In [21]:
%%sql
SELECT nome, nomecategoria, preco
FROM produto 
WHERE nomecategoria = 'Categoria 2'
ORDER BY preco DESC
LIMIT 10;

 * sqlite:///banco_de_dados.db
Done.


nome,nomecategoria,preco
Produto 11,Categoria 2,993.44
Produto 86,Categoria 2,991.4
Produto 91,Categoria 2,918.52
Produto 31,Categoria 2,895.53
Produto 76,Categoria 2,869.35
Produto 66,Categoria 2,846.68
Produto 71,Categoria 2,805.6
Produto 16,Categoria 2,766.33
Produto 51,Categoria 2,722.91
Produto 36,Categoria 2,590.59


### 6. Quantos produtos a empresa comercializa?

In [22]:
%%sql
SELECT COUNT(produtoid) AS total_de_produtos
FROM produto

 * sqlite:///banco_de_dados.db
Done.


total_de_produtos
100


### 7. Quais os valores: mínimo, máximo, a média e o total de produtos da categoria 2?

In [23]:
%%sql
SELECT nomecategoria,
MIN(preco) AS valor_minimo,
MAX(preco) AS valor_maximo,
ROUND(AVG(preco), 2) AS media,
COUNT(preco) AS total_produtos
FROM produto
WHERE nomecategoria = 'Categoria 2';

 * sqlite:///banco_de_dados.db
Done.


nomecategoria,valor_minimo,valor_maximo,media,total_produtos
Categoria 2,96.25,993.44,588.84,20


### 8. Quais os produtos da categoria 3 têm preços superiores a R$ 350,00?

In [24]:
%%sql
SELECT nome, preco, nomecategoria
FROM produto
WHERE nomecategoria = 'Categoria 3' AND preco > 350.00

 * sqlite:///banco_de_dados.db
Done.


nome,preco,nomecategoria
Produto 17,358.08,Categoria 3
Produto 22,584.69,Categoria 3
Produto 47,968.15,Categoria 3
Produto 57,990.91,Categoria 3
Produto 67,932.19,Categoria 3
Produto 72,751.92,Categoria 3
Produto 77,456.47,Categoria 3
Produto 82,857.05,Categoria 3
Produto 87,790.92,Categoria 3
Produto 92,751.29,Categoria 3


### 9. Quais produtos da categoria 3 tem valores igual ao maior do que R$ 600,00?

In [25]:
%%sql
SELECT nomecategoria, nome, preco
FROM produto
WHERE nomecategoria = 'Categoria 3' AND preco >= 600.00

 * sqlite:///banco_de_dados.db
Done.


nomecategoria,nome,preco
Categoria 3,Produto 47,968.15
Categoria 3,Produto 57,990.91
Categoria 3,Produto 67,932.19
Categoria 3,Produto 72,751.92
Categoria 3,Produto 82,857.05
Categoria 3,Produto 87,790.92
Categoria 3,Produto 92,751.29


### 10. Quais as cinco maiores médias de preço por categoria de produto?

In [26]:
%%sql
SELECT p.nome, p.nomecategoria, ROUND(AVG(p.preco), 2) AS media_preco
FROM produto AS p
GROUP BY p.nome, p.nomecategoria
ORDER BY media_preco DESC
LIMIT 5;

 * sqlite:///banco_de_dados.db
Done.


nome,nomecategoria,media_preco
Produto 11,Categoria 2,993.44
Produto 86,Categoria 2,991.4
Produto 57,Categoria 3,990.91
Produto 47,Categoria 3,968.15
Produto 88,Categoria 4,958.69


### 11. Qual a média de faturamento por categoria de produto? 

In [27]:
%%sql
SELECT p.nomecategoria, ROUND(AVG(v.faturamento), 2) AS media_faturamento
FROM produto AS p
JOIN vendas AS V ON v.produtoid = p.produtoid
GROUP BY nomecategoria;

 * sqlite:///banco_de_dados.db
Done.


nomecategoria,media_faturamento
Categoria 1,2445.82
Categoria 2,2547.16
Categoria 3,2501.23
Categoria 4,2521.71
Categoria 5,2505.9


### 12. Qual o faturamento total por cidade do cliente? 

In [28]:
%%sql
SELECT c.cidadecliente, ROUND(SUM(v.faturamento), 2) AS total_faturamento
FROM cliente AS c
JOIN vendas AS v ON v.clienteid = c.clienteid
GROUP BY c.cidadecliente;

 * sqlite:///banco_de_dados.db
Done.


cidadecliente,total_faturamento
Buenos Aires,5049234.62
Ceará,4596261.78
New York,4735172.63
Rio de Janeiro,5061534.85
São Paulo,5603347.08


### 13. Qual o faturamento médio nos dias úteis? 

In [29]:
%%sql
SELECT d.diadasemana, ROUND(AVG(v.faturamento), 2) AS media_faturamento
FROM data AS d
JOIN vendas AS v ON v.dataid = d.dataid
WHERE TRIM(d.diadasemana) NOT IN('Sunday', 'Saturday') 
GROUP BY d.diadasemana
ORDER BY media_faturamento DESC;

 * sqlite:///banco_de_dados.db
Done.


diadasemana,media_faturamento
Monday,2574.91
Tuesday,2531.75
Wednesday,2493.11
Friday,2468.17
Thursday,2435.96


### 14. Qual o valor total das vendas e do custo de frete por país do cliente e categoria de produto no ano de 2024?

In [30]:
%%sql
SELECT d.data, c.paiscliente, p.nomecategoria, d.ano, SUM(v.quantidadevendida) AS total_vendas,
ROUND(SUM(v.custofrete), 2) AS total_frete
FROM cliente AS c
JOIN vendas AS v ON v.clienteid = c.clienteid
JOIN produto AS p ON v.produtoid = p.produtoid
JOIN data AS d ON v.dataid = d.dataid
WHERE d.ano = 2024
GROUP BY c.paiscliente, p.nomecategoria, d.ano
ORDER BY c.paiscliente;

 * sqlite:///banco_de_dados.db
Done.


data,paiscliente,nomecategoria,ano,total_vendas,total_frete
2024-01-09,Argentina,Categoria 1,2024,3991,7736.72
2024-01-05,Argentina,Categoria 2,2024,4222,10113.15
2024-01-02,Argentina,Categoria 3,2024,3819,8966.76
2024-01-01,Argentina,Categoria 4,2024,3870,10137.66
2024-01-01,Argentina,Categoria 5,2024,4452,11466.04
2024-01-03,Brasil,Categoria 1,2024,11953,33415.14
2024-01-02,Brasil,Categoria 2,2024,10804,27615.69
2024-01-01,Brasil,Categoria 3,2024,11479,28435.43
2024-01-03,Brasil,Categoria 4,2024,11784,31540.25
2024-01-02,Brasil,Categoria 5,2024,13100,30719.17


### 15. Qual o total de vendas e custo de frete por país do cliente e categoria de produto no ano de 2024 quando o custo total de frete for maior que 30.000?

In [31]:
%%sql
SELECT d.data, c.paiscliente, p.nomecategoria, SUM(v.quantidadevendida) AS total_vendas, 
ROUND(SUM(v.custofrete), 2) AS total_frete
FROM cliente AS c
JOIN vendas AS v ON v.clienteid = c.clienteid
JOIN produto AS p ON v.produtoid = p.produtoid
JOIN data AS d ON v.dataid = d.dataid
WHERE ano = 2024
GROUP BY c.paiscliente, p.nomecategoria
HAVING SUM(v.custofrete) > 30000
ORDER BY c.paiscliente, p.nomecategoria;

 * sqlite:///banco_de_dados.db
Done.


data,paiscliente,nomecategoria,total_vendas,total_frete
2024-01-03,Brasil,Categoria 1,11953,33415.14
2024-01-03,Brasil,Categoria 4,11784,31540.25
2024-01-02,Brasil,Categoria 5,13100,30719.17


In [32]:
# Feche a conexão com o banco de dados
cnn.close()